In [2]:
import pandas as pd
import numpy as np
#--------------------------------------------------
# Cleaning TCGA phenotype data
#--------------------------------------------------
# Read data
clinical_phenotype_df = pd.read_csv(
    "TCGA_data/phenotype_data", 
    sep="\t"
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
print(f"Original shape: {clinical_phenotype_df.shape}")

# Filter patients: Adults (18+) with Primary Tumors
clinical_phenotype_df = clinical_phenotype_df[
    (clinical_phenotype_df['age_at_index.demographic'] >= 18) &  
    (clinical_phenotype_df['sample_type.samples'] == 'Primary Tumor')
]

# Define missing value indicators
missing_values = [
    'not reported', 'Not Reported', 'NOT REPORTED',
    'unknown', 'Unknown', 'UNKNOWN',
    'not available', 'Not Available', 'N/A', 'NA', 'n/a',
    'not applicable', 'Not Applicable',
    '--', '---', 'null', 'NULL', 'None',
    '[Not Available]', '[Not Applicable]', '[Unknown]',
    'Not Evaluated', 'not evaluated',
    '', ' ', '  '
]

# Replace with NaN
clinical_phenotype_df.replace(missing_values, np.nan, inplace=True)

# Remove columns with >90% missing
threshold = 0.1 * len(clinical_phenotype_df)
clinical_phenotype_df = clinical_phenotype_df.dropna(axis=1, thresh=threshold)

print(f"Shape before treatment cleaning: {clinical_phenotype_df.shape}")

# Cleaning up treatment types
clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'].replace("['yes', 'yes']", 'RT/TMZ', inplace=True)
clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'].replace("['no', 'no']", 'no treatment', inplace=True)
clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'].replace("['no', 'yes']", 'RT', inplace=True)
clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'].replace("['yes', 'no']", 'TMZ', inplace=True)

# Remove rows where treatment is still NaN (was "not reported")
clinical_phenotype_df = clinical_phenotype_df.dropna(
    subset=['treatment_or_therapy.treatments.diagnoses']
)

print(f"Shape after removing 'not reported' treatments: {clinical_phenotype_df.shape}")
print(f"\nUnique treatment values:")
print(clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'].unique())
print(f"\nTreatment distribution:")
print(clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'].value_counts())

RT_df = clinical_phenotype_df[clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'] == 'RT']
RT_and_TMZ_df = clinical_phenotype_df[clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'] == 'RT/TMZ']
TMZ_df = clinical_phenotype_df[clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'] == 'TMZ']
No_treatment_df = clinical_phenotype_df[clinical_phenotype_df['treatment_or_therapy.treatments.diagnoses'] == 'no treatment']

# Save cleaned data
output_path = '/Users/lynettewilson/Desktop/mRNA_Cancer_Prediction_Model/TCGA_data/clean_phenotype_data.csv'
clinical_phenotype_df.to_csv(output_path, index=False)

print(f"\nFinal shape: {clinical_phenotype_df.shape}")
print("Data saved successfully!")

Original shape: (636, 82)
Shape before treatment cleaning: (600, 69)
Shape after removing 'not reported' treatments: (600, 69)

Unique treatment values:
['RT/TMZ' "['not reported', 'yes']" 'RT' 'no treatment'
 "['not reported', 'not reported']" "['yes', 'not reported']" 'TMZ']

Treatment distribution:
treatment_or_therapy.treatments.diagnoses
RT/TMZ                              450
no treatment                         45
RT                                   27
['not reported', 'not reported']     26
TMZ                                  23
['yes', 'not reported']              15
['not reported', 'yes']              14
Name: count, dtype: int64

Final shape: (600, 69)
Data saved successfully!


/var/folders/cq/3b5dz8vx1wn__sk129npctq40000gn/T/ipykernel_2976/1362187502.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clinical_phenotype_df.replace(missing_values, np.nan, inplace=True)
/var/folders/cq/3b5dz8vx1wn__sk129npctq40000gn/T/ipykernel_2976/1362187502.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on t

In [3]:
#--------------------------------------------------
# Cleaning CGGA clinical data
#--------------------------------------------------

clinical_cgga_df = pd.read_csv(
    "CGGA_data/CGGA_clinical.txt", 
    sep="\t")

print(clinical_cgga_df.shape)

clinical_cgga_df = clinical_cgga_df[(clinical_cgga_df['Age'] >= 18) 
                                    &(clinical_cgga_df['Histology'] == 'GBM')
                                    &(clinical_cgga_df['IDH_mutation_status'] == 'Wildtype')]
print(clinical_cgga_df.shape)
clinical_cgga_df.replace(missing_values, np.nan, inplace=True)
print(clinical_cgga_df.shape)
output_path_2 = '/Users/lynettewilson/Desktop/mRNA_Cancer_Prediction_Model/TCGA_data/clean_cgga_clinical_data.csv'
clinical_cgga_df.to_csv(output_path_2, index=False)





(702, 9)
(139, 9)
(139, 9)


In [3]:
df = pd.read_csv("TCGA_data/combined_clinical_RNAseq.csv", sep=",")
df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,combined_clinical_RNAseq
ID,Histology,Grade,Gender,Age,OS,Censor,IDH_mutation_status,1p19q_codeletion_status,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,RTN4RL2,C16orf13,C16orf11,FGFR1OP2,TSKS,ATRX,PMM2,LOC100272146,ASS1,NCBP1,ZNF709,ZNF708,RBM14,NCBP2,DISC1,CAMK1,RPL37,SPR,ZNF700,ZNF707,CAMK4,ZNF704,LOC339240,GOLGA6B,RNF115,RNF112,ZC3H14,SPN,HMGCLL1,NACAP1,LRRTM1,GRIN1,DHX8,DHX9,LOC441204,TCOF1,LRRTM3,NUP98,XPC,SLC12A2,GRINA,LRRTM2,NUP93,HAP1,KLRA1,CAMKV,SPPL3,GOLIM4,ZNF646,OPA3,OPA1,RHEB,RAB40C,RAB40B,RAB40A,COL7A1,MDP1,GTSE1,FAM183A,ARFRP1,OVCH1,FAM183B,PSAP,KIAA0831,SPPL2A,SLC46A1,TGFBR2,ITGA8,ITGA9,STK40,MYO3B,ATP2A1,ATP2A2,ATP2A3,ITGA1,ITGA2,ITGA3,ITGA4,ITGA5,ITGA6,ITGA7,TRHR,LOC100132288,DENND4A,DENND4B,DENND4C,SWAP70,RARRES1,LOC100132287,PHLDA3,PHLDA2,TACO1,PMM1,GAP43,NENF,SQLE,MRPL28,LOC100289341,FBXL13,FBXL12,FBXL15,CYP27B1,FBXL17,FBXL16,FAM169A,FAM116B,FAM116A,FAM169B,RALGAPA2,GHDC,H19,SDK2,GRP,TPRN,THSD4,KIAA0895L,CHST9,CHST8,COL4A5,COL4A4,COL4A3,COL4A2,COL4A1,SLC35D2,CHST1,ZNF879,CHST3,CHST2,CHST5,CHST4,CHST7,CHST6,ITGAX,BCL2A1,GPSM2,HLCS,NOG,LIPG,ITGAV,HIBCH,ZNF703,ITGAL,ITGAM,LOC286359,HLF,NOV,FAM134C,FAM134B,FAM134A,SERPING1,SMAD9,TRERF1,LOC653501,LRPAP1,LQK1,SMAD6,SMAD7,GOLPH3L,SMAD1,AFTPH,SMAD3,MUS81,AMMECR1L,IL1RAPL1,IGF2R,TMPRSS11D,TMPRSS11F,ZC3H10,RAD21L1,IL1RAPL2,SLC36A3,CKS1B,SLC36A1,ST13,RPL31P11,C14orf115,ZC3H15,TMEM213,C14orf118,ARTN,ART1,HRH3,ART3,HERC2P2,ART5,ART4,RNF111,TCEA2,TCEA3,TCEA1,MGC57346,COX4I2,COX4I1,BFAR,ZC3H18,TMEM212,SNORD115-13,TCEB3B,SIPA1,CRTAM,EDA,CCDC116,CADPS2,CCDC114,CCDC115,CCDC112,CCDC113,CCDC110,CCDC111,LEO1,ATP1A4,ATP1A3,ATP1A2,ATP1A1,KGFLP2,MGRN1,TCP1,CRTAP,CBLC,SPHK1,DPT,CUEDC1,CUEDC2,CBLB,CPEB4,CPEB1,ZNF45,CPEB3,CPEB2,BEGAIN,RBMXL3,OSGEP,ABAT,LOC100129637,ZNF780A,ZNF780B,TLN2,FRG1,RHEBL1,ZNF41,IFT80,FBRS,ZNF414,IFT88,IL10RA,POU5F1B,SP1,SLC27A4,SP2,OLA1,PNOC,CNTF,MCM10,PDIA5,SP4,TMEM38B,THUMPD3,IGFBPL1,SP5,ATG4C,ATG4B,ATG4A,THUMPD2,LOC441208,ATG4D,PHOX2A,PHOX2B,LRGUK,MPHOSPH8,UPK2,CFH,CFI,PNP,ANKFY1,CFB,MLLT4,CREBBP,PNN,PHYHIPL,ATP6V0E1,ATP6V0E2,RNF128,CFP,AAAS,HAMP,ZNF394,SIDT1,SERPINA10,SIDT2,SNORD115-7,TAF8,SIX4,LOC283070,SIX6,PM20D1,MR1,MGC4473,SIX3,WASH3P,CNPY4,CNPY3,CNPY2,CNPY1,NODAL,SYNJ1,KNDC1,HMG20A,HMG20B,PHOSPHO2,AAGAB,C4orf19,C3orf79,SLC7A6OS,C3orf77,C3orf75,C3orf74,C4orf10,C3orf72,C3orf71,C3orf70,ZNF285,MRO,ZFP82,CHFR,EFHA1,RTBDN,EFHA2,AAA1,GPR75,NUDT9P1,SYPL2,RFX8,ZNF222,RFX4,OR2C3,AVPR2,RFX7,NRXN3,GPR77,NRXN1,RFX3,RFPL2,VPS37D,APC,TMEM179B,ABCE1,UQCRC2,XPNPEP1,XPNPEP2,TIGD4,ZNHIT3,GGCX,PAPLN,ADNP,TMEM218,STXBP6,KHDRBS1,RUNDC2C,KHDRBS3,GGCT,VPS37C,STAC2,LOC283404,ZNF395,STAC3,TBL1Y,LHCGR,TMEM211,OR1Q1,TMEM127,GPR78,TRPA1,VPS37B,TTC7A,UQCRHL,ZBED4,C11orf2,C11orf1,DPYSL5,DPYSL2,DPYSL3,OSTM1,RUNX2,C11orf9,IGFBP2,IGFBP3,IGFBP1,BOP 1.00,IGFBP7,IGFBP4,IGFBP5,SKP2,TTC4,CYP2R1,CKLF,ZNF193,C15orf26,C15orf27,C15orf24,GJA3,APLNR,C15orf23,C15orf21,GJA8,CYP8B1,C15orf28,C15orf29,STMN1,S1PR2,KDM3B,STK17A,KDM3A,LOC401387,S1PR1,ZNF642,TMC4,MYO3A,TMC6,KIAA1958,APIP,TMC1,GPR15,TMC3,P